# Testbed Analysis Notebook

## Notebook Parameters

In [ ]:
LOGS = [
    '<date>/<datetime>', # path in logs directory
]

ONLY_ENTRIES = []
SKIP_ENTRIES = []

# entry: (head_num, tail_num)
DROP_FROM_ENTRIES = {}

CORRECT_CLOCK_OFFSET = True

PLOT_TAG = ""

LATENCY_PLOT_MAX = None         # [ms]
LATENCY_PLOT_MIN = None         # [ms]

LOADS_DISPLAYED = {
    'UL': [0, 1, 3],
    'DL': [0, 2, 3],
    'EE': [0, 1, 2, 3]
}

ANTENNAS_COORDINATES = [] # [Anntena 1 (lat, lon), Antenna 2 (lat, lon)]
HANDOVER_TIMES = {} # {'<datetime>': [['<date> <time>', '<date> <time>']]}

LABEL = {}

COLORS = {
    'UL': '#264653',
    'DL': '#e76f51',
    'EE': '#68985d',
    'Mean': '#e9c46a',
    'Trajectory': '#264653',
    'Antenna1_Light':'#ed947e',
    'Antenna1_':'#e78851',
    'Antenna1': '#e76f51',
    'Antenna2': '#e9c46a',
    'HOs': '#2a9d8f',
    # 'Antenna2': (38 / 255.0, 70 / 255.0, 83 / 255.0, 1),  # blue as an example
    # 'Antenna1': (231 / 255.0, 111 / 255.0, 81 / 255.0, 1),  # orange as an example
}

#7f7f7f grey
COLOR_PALETE = ['#e76f51', '#6fb0dc', '#bf4342',  '#000000', '#264653', '#008a76', '#e9c46a', '#007a99', '#90be6d', '#8c564b', '#9467bd', '#cf8ba9']
LINE_STYLES = ["--", "-", ".-"]

## Setup

### Imports & Constants


In [ ]:
from pathlib import Path
from typing import Tuple
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import ScalarFormatter
from datetime import datetime
import numpy as np

import yaml
import pint
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.stats import norm

ureg = pint.UnitRegistry()

LOGS_DIR = Path().absolute().parent / "logs"
LOGS_DIR.mkdir(exist_ok=True)

PLOTS_DIR = Path().absolute().parent / "plots"
PLOTS_DIR.mkdir(exist_ok=True)

### Utility Functions

In [ ]:
def get_data(*dirs: str) -> Tuple[pd.DataFrame, dict]:
    assert dirs, "No data directories provided"

    data = {}
    meta = {}
    for dir in map(lambda name: LOGS_DIR / name, dirs):
        
        with open(dir / "flags.yml") as f:
            flags = yaml.load(f, yaml.CLoader)

        for test in flags:
            entry = test["datetime"] 

            if ONLY_ENTRIES and entry not in ONLY_ENTRIES:
                print(f"Skipping {entry}")
                continue
            if entry in SKIP_ENTRIES:
                print(f"Skipping {entry}")
                continue

            df = pd.read_csv(dir / test["filename"], index_col=False)
            df[["t1", "t2", "t3", "t4", "e1", "e2", "e3", "e4"]] /= 1e6  # [ms]
           
            if entry in DROP_FROM_ENTRIES:
                df = df.tail(df.shape[0] - DROP_FROM_ENTRIES[entry][0])
                df = df.head(df.shape[0] - DROP_FROM_ENTRIES[entry][1])
          
            data[entry] = df
            meta[entry] = {
                "group": dir.name,
                "case": test["case"],
                "rate": test["rate"],
                "size": test["size"],
                "mobility": test["mobility"],
                "features": test["features"],
                "datetime": test["datetime"],
            }

    return data, meta

def sensor_time(df: pd.DataFrame) -> pd.Series:
    time = df["t1"] + df["e1"] if CORRECT_CLOCK_OFFSET else df["t1"]
    return (time) - (time[0])

def clock_offset_ul(df: pd.DataFrame) -> pd.Series:
    return df["e2"] - df["e1"]

def clock_offset_dl(df: pd.DataFrame) -> pd.Series:
    return df["e4"] - df["e3"]

def clock_offset_ee(df: pd.DataFrame) -> pd.Series:
    return df["e4"] - df["e1"]

def latency_ul(df: pd.DataFrame) -> pd.Series:
    l = df['t2'] - df['t1']
    if CORRECT_CLOCK_OFFSET:
        l += clock_offset_ul(df)
    return l

def latency_dl(df: pd.DataFrame) -> pd.Series:
    l = df['t4'] - df['t3']
    if CORRECT_CLOCK_OFFSET:
        l += clock_offset_dl(df)
    return l

def latency_ee(df: pd.DataFrame) -> pd.Series:
    l = df['t4'] - df['t1']
    if CORRECT_CLOCK_OFFSET:
        l += clock_offset_ee(df)
    return l

def info_label(meta: dict) -> str:
    return "\n".join([
        datetime.strptime(meta["group"], "%y%m%d_%H%M").strftime(f"TC{meta['case']}, G%H%M"),
        datetime.strptime(meta["datetime"], "%y%m%d_%H%M").strftime("(%y-%m-%d %H:%M)"),
    ])

def convert_to_timestamp(timestamp_str):
    truncated_timestamp_str = timestamp_str[:26]  # Keeps up to microseconds
    timestamp_format = "%Y-%m-%d %H:%M:%S.%f"

    # Parse the truncated timestamp
    timestamp = datetime.strptime(truncated_timestamp_str, timestamp_format)

    # Extract the nanoseconds part
    nanoseconds_part = int(timestamp_str[26:])

    # Convert datetime to Unix timestamp in seconds and add nanoseconds
    unix_timestamp_seconds = timestamp.timestamp()
    total_nanoseconds = int(unix_timestamp_seconds * 1e9) + nanoseconds_part

    return total_nanoseconds / 1e6

def find_handover_coordinates(df, target_values, corrected):
    start_coordinates = (0, 0)
    end_coordinates = (0, 0)
    if 't4' not in df.columns or 'e4' not in df.columns or 'lat' not in df.columns or 'lon' not in df.columns:
        print("Required columns not found in the CSV file.")
        return start_coordinates, end_coordinates

    df['computed_value'] = df['t4'] + df['e4'] if corrected else df['t4']
    
    # Find the row with the closest match to the target_value
    start_idx = (df['computed_value'] - convert_to_timestamp(target_values[0])).abs().idxmin()
    closest_start_row = df.iloc[start_idx]
    exact_match_start_row = df[df['computed_value'] == convert_to_timestamp(target_values[0])]
    
    if not exact_match_start_row.empty:
        closest_start_row = exact_match_start_row.iloc[0]
        start_idx = exact_match_start_row.index[0]

    end_idx = (df['computed_value'] - convert_to_timestamp(target_values[1])).abs().idxmin()
    closest_end_row = df.iloc[end_idx]
    exact_match_end_row = df[df['computed_value'] == convert_to_timestamp(target_values[1])]

    if not exact_match_end_row.empty:
        closest_end_row = exact_match_end_row.iloc[-1]
        end_idx = exact_match_start_row.index[-1]

    return (closest_start_row['lon'], closest_start_row['lat'], start_idx), (closest_end_row['lon'], closest_end_row['lat'], end_idx)

def distance(a, b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def get_color_index(code):
    code = str(code)

    if len(code) != 4:
        raise ValueError("Code must be in the format 'XXXX' X is a digit.")

    numeric_part = int(code[1:])
    hundreds_digit = numeric_part // 100
    tens_digit = numeric_part // 10 % 10  # Extract the tens digit (Y)
    ones_digit = numeric_part % 10        # Extract the ones digit (Y)


    # Calculate index for X040-X063 pattern
    if tens_digit in range(4, 7) and ones_digit in range(0, 4):
        return tens_digit * 4 + ones_digit - 16

    # Calculate index for X103, X113, X123, X133, X203, X213, X223, X233 pattern
    if ones_digit == 3 and tens_digit in range(0, 4) and hundreds_digit in range(1, 4):
        return (numeric_part // 100 - 1) * 4 + tens_digit

    if ones_digit in range(3, 5) and tens_digit in range(0, 8) and hundreds_digit in range(0, 5):
        return (numeric_part // 100 - 1) * 4 + tens_digit - ones_digit 

    if tens_digit in range(7, 10) and ones_digit in range(0, 4):
        return tens_digit * 4 + ones_digit - 28
    
    # Calculate index for X010, X023, X032 pattern
    if tens_digit in range(1, 4) and ones_digit in range(0, 4):
        return tens_digit * 4 + ones_digit - 16
    else:
        raise ValueError("Invalid code format.")


def get_row_identifier(case_number):
    case_str = str(case_number)
    prefix = int(case_str[:3])
    
    if int(case_str[1]) in range(1, 3) and int(case_str[2]) in range(0, 4):
        # For these cases, use the first three digits as the row identifier
        return int(case_str[:2])
    else:
        # For all other cases, use the first two digits as the row identifier
        return prefix

def sort_key(item):
    case_str = str( item[1]["case"])
    
    # Use the first digit for primary sorting, and the last three digits for secondary sorting
    if int(case_str[1]) in range(1, 3) and int(case_str[2]) in range(0, 4):
        # Use the first digit for primary sorting, and last digit for secondary sorting
        return (int(case_str[1:2]), int(case_str[0]), int(case_str[3]))
    else:
        return (int(case_str[:3]), int(case_str[3:]))
    
def return_label(name):
    if name in LABEL:
     return LABEL[name]
    
    return f"Test case: {name}"

def is_within_handover(df, idx, handover_periods, idxs, corrected_time):
    """Check if the given timestamp is within any handover period."""
    timestamp = df['t4'].iloc[idx] + df['e4'].iloc[idx] if corrected_time else df['t4'].iloc[idx]
    for (start, end), (start_idx, end_idx) in zip(handover_periods, idxs):
        if (convert_to_timestamp(start) <= timestamp <= convert_to_timestamp(end)) or start_idx <= idx <= end_idx:
            return True
    return False

## Plots

In [ ]:
DATA, META = get_data(*LOGS)

### Latency (individual packets and CDF)

In [ ]:
def main():
    
    fw, ncols = 6, 3
    fh, nrows = 4, len(META)
    fig = plt.figure(figsize=(fw*ncols, fh*nrows), facecolor="w", edgecolor="k")
    axs = fig.subplots(nrows, ncols, sharey='row').reshape(nrows, ncols)
    
    items = list(META.items())
    items.sort(key=lambda x: x[1]["case"])

    for i, (entry, meta) in enumerate(items):
        df = DATA[entry]
        for j, title in enumerate(["UL packets", "DL packets", "CDFs"]):
            ax = axs[i, j]

            ax.set_title(title, fontweight="bold")
            axs[i, 0].set_ylabel(f"{info_label(meta)}\n{return_label(str(meta['case']))}\n\nLatency [ms]")

            if LATENCY_PLOT_MAX or LATENCY_PLOT_MIN:
                ax.set_ylim(LATENCY_PLOT_MIN, LATENCY_PLOT_MAX)
            ax.grid()

            if j == 0:
                ul = latency_ul(df)
                ax.plot(ul)
                ax.set_xlabel("Packet number")
            if j == 1:
                dl = latency_dl(df)
                ax.plot(dl)
                ax.set_xlabel("Packet number")
            if j == 2:

                for label, xs in zip(["UL", "DL"], [ul, dl]):
                    xs = xs.sort_values()
                    cdf = xs.rank(pct=True) * 100
                    ax.plot(cdf, xs, "-^", label=label, markersize=4)

                for label, xs in zip(["UL", "DL"], [ul, dl]):
                    p = xs.quantile(0.5)
                    ax.axhline(p, color="k", linestyle='--')
                    ax.annotate(f'{p:.02f}', xy=(2, p + 2), color='k')

                    p = xs.quantile(0.95)
                    ax.axhline(p, color="k", linestyle='-')
                    ax.annotate(f'{p:.02f}', xy=(2, p + 2), color='k')
            
                ax.axvline(50, color="k", linestyle='--')
                ax.axvline(95, color="k")
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    filename = f"{PLOT_TAG}__latency" if PLOT_TAG else "latency"
    fig.savefig(PLOTS_DIR / f"{filename}.png")

main()

### Clock Offset (individual packets)

In [ ]:
def main():

    items = list(META.items())
    items.sort(key=lambda x: x[1]["case"])

    fw, ncols = 6, 2
    fh, nrows = 4, len(META)
    fig = plt.figure(figsize=(fw*ncols, fh*nrows), facecolor="w", edgecolor="k")
    axs = fig.subplots(nrows, ncols, sharey='row').reshape(nrows, ncols)

    for i, (entry, meta) in enumerate(items):
        df = DATA[entry]
        for j, title in enumerate(["Server ahead of Sensor (UL)", "Vehicle ahead of Server (DL)"]):
            ax = axs[i, j]
            
            ax.set_title(title, fontweight="bold")
            axs[i, 0].set_ylabel(f"{info_label(meta)}\n\nClock Offset [ms]")
            
            ax.axhline(0, color="r")
            ax.grid()

            if j == 0:
                ax.plot(clock_offset_ul(df))
                ax.set_xlabel("Packet number")
                # print("Clock offset UL: ", entry)
                # print(clock_offset_ul(df).describe())
            if j == 1:
                ax.plot(clock_offset_dl(df))
                ax.set_xlabel("Packet number")
                # print("Clock offset DL: ", entry)
                # print(clock_offset_dl(df).describe())

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    filename = f"{PLOT_TAG}__clock" if PLOT_TAG else "clock"
    fig.savefig(PLOTS_DIR / f"{filename}.png")


main()

### Compare Latency Between Test Cases

In [ ]:
def colors_legend(index): 
    return mlines.Line2D([], [], marker='o', markeredgecolor=COLOR_PALETE[index], markerfacecolor=COLOR_PALETE[index], color='none')

def plot_latency_data(data, latency_func, ax, x_counter, marker, linewidth):
    latency = latency_func(data)
    quantile_value = latency.quantile(0.95)
    quantile_max_value = latency.quantile(0.99)
    mean_value = latency.mean()

    ax.scatter([x_counter], [quantile_max_value], marker=marker, s=50, linewidth=linewidth, color=COLOR_PALETE[2], zorder=3)
    ax.scatter([x_counter], [quantile_value], marker=marker, s=50, linewidth=linewidth, color=COLOR_PALETE[5], zorder=3)
    ax.scatter([x_counter], [mean_value], marker=marker, s=50, linewidth=linewidth, color=COLOR_PALETE[6], zorder=3)

    return mlines.Line2D([], [], marker=marker, linewidth=2, color='none', zorder=3, markeredgecolor='black')


def main():
    items = list(META.items())
    items.sort(key=lambda x: x[1]['case'])    
    

    for lat in ['ul', 'dl', 'ee']:
        fig, ax = plt.subplots(figsize=(1.8 * len(items), 8), facecolor="w", edgecolor="k")
        ax.margins(0.45)
        xticks=[]
        x_positions = []
        x_counter = 0
        for entry, meta in items:
            df = DATA[entry]
            
            shape = plot_latency_data(data=df, latency_func=eval('latency_'+ lat), ax= ax, x_counter=x_counter, marker='x', linewidth=3)

            xticks.append(info_label(meta))
            x_positions.append(x_counter)
            x_counter += 1

        # y-ticks
        _, y_max = ax.get_ylim()
        ax.set_yticks(np.round(np.linspace(0, y_max, num=15)))
        ax.set_ylabel('Latency [ms]', fontweight="bold", fontsize="14")

        # x-ticks
        ax.set_xticks(x_positions)
        ax.set_xticklabels(xticks, rotation=90, ha='right')

        ax.set_title(f"Comparing Test Cases \n {lat.upper()} Latency", fontweight='bold')
        
        ax.grid()
        ax.legend([colors_legend(2), colors_legend(5), colors_legend(6)], ['99% Latency', '95% Latency', 'Mean Latency'], loc='upper right', bbox_to_anchor=(1.38, 1))
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])

        filename = f"{PLOT_TAG}__comparison" if PLOT_TAG else "comparison"
        fig.savefig(PLOTS_DIR / f"{filename}.png")

if __name__ == '__main__':
    main()

### Compare Latency Between Test Cases (Alternative)

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: x[1]['case'])    
    
    fig, ax = plt.subplots(figsize=(2 * len(items), 8), facecolor="w", edgecolor="k")
    
    xticks=[]
    x_positions = []
    x_counter = 0

    for entry, meta in items:
        df = DATA[entry]

        ul_latencies = latency_ul(df)
        dl_latencies = latency_dl(df)

        ul_quantile_value = ul_latencies.quantile(0.95)
        dl_quantile_value = dl_latencies.quantile(0.95)

        # Plot regular Uplink and Downlink latencies first
        ax.scatter([x_counter] * len(ul_latencies), ul_latencies, marker="s", s=50, label='Uplink', color='#2a9d8f')
        ax.scatter([x_counter + 1] * len(dl_latencies), dl_latencies, marker="s", s=50, label='Downlink', color='#e9c46a')
        
        # Plot the red points afterward to make sure they are on top
        ax.scatter([x_counter], [ul_quantile_value], marker="s", s=50, color='#e76f51', zorder=3)
        ax.scatter([x_counter + 1], [dl_quantile_value], marker="s", s=50, color='#e76f51', zorder=3)
        
        xticks.append(info_label(meta))
        x_positions.append(x_counter)
        x_counter += 3

    # y-ticks
    y_min, y_max = ax.get_ylim()
    ax.set_yticks(np.linspace(y_min, y_max, num=30))
    ax.set_ylabel('Latency', fontweight="bold", fontsize="14")

    # x-ticks
    ax.set_xticks(x_positions)
    ax.set_xticklabels(xticks, rotation=30, ha='right')
    ax.set_xlabel('Test Case', fontweight="bold",  fontsize="14")

    ax.set_title(f"Comparing Test Cases: {95:.0f}% Percentile", fontweight='bold')
    
    ax.grid()
    ax.legend(['Uplink', 'Downlink'], loc='upper right')

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    filename = f"{PLOT_TAG}__comparison" if PLOT_TAG else "comparison"
    fig.savefig(PLOTS_DIR / f"{filename}.png")

if __name__ == '__main__':
    main()

### Latency over Sensor's Time (per Test Case)

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=sort_key)
    
    unique_prefixes = sorted(set(get_row_identifier(meta['case']) for _, meta in items))

    nrows = len(unique_prefixes)
    ncols = max([list(map(int, [get_row_identifier(meta['case']) for _, meta in items])).count(prefix) for prefix in unique_prefixes])

    fw, fh = 8 * ncols, 6 * nrows
    fig, axs = plt.subplots(nrows, ncols, figsize=(fw, fh), facecolor='w', edgecolor='k')

    # Make sure axs is always a 2D array
    if not isinstance(axs, np.ndarray):
        axs = np.array([[axs]])

    if nrows == 1:
        axs = axs.reshape(nrows, -1)
    if ncols == 1:
        axs = axs.reshape(-1, ncols)

    current_row = 0
    current_col = 0
    last_prefix = None

    for entry, meta in items:
        legend_handles_colors = []
        current_prefix = get_row_identifier(meta['case'])
        
        if last_prefix is not None and current_prefix != last_prefix:
            current_row += 1
            current_col = 0
        
        ax = axs[current_row, current_col]

        df = DATA.get(entry, [])
        ul = latency_ul(df)
        dl = latency_dl(df)

        ax.set_title(f"{info_label(meta)}", fontweight="bold", fontsize="14")
        ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")
        ax.set_xlabel(f"Sensor's Time [s]", fontweight="bold", fontsize="14")


        if LATENCY_PLOT_MAX or LATENCY_PLOT_MIN:
            ax.set_ylim(LATENCY_PLOT_MIN, LATENCY_PLOT_MAX)

        ax.grid()

        for label, xs in zip(["UL", "DL"], [ul, dl]):
            time = sensor_time(df)/1e3

            scatter = ax.scatter(time, xs, label=label, color=COLORS[label], alpha=0.4)

            # lines on x and y axis
            p = xs.quantile(0.5)
            ax.axhline(p, linestyle='--', linewidth=1.5, color=COLORS[label])

            p = xs.quantile(0.95)
            ax.axhline(p, linestyle='-', linewidth=1.5, color=COLORS[label])
            legend_handles_colors.append(scatter)
        
        legend_handles_shapes = [
            Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
            Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
        ]
        ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper right') 

        current_col += 1
        last_prefix = current_prefix

    fig.suptitle("Latency over Sensor's Time (per Test Case)", fontweight='bold', fontsize=24)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_over_time" if PLOT_TAG else "latency_over_time"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### CDFs of different message sizes and frequencies

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: str(x[1]["case"]))
    
    unique_prefixes = sorted(set(str(meta['case'])[:3] for _, meta in items))
    nrows = len(unique_prefixes)
    ncols = 3  # UL, DL, and EE for each unique prefix

    fig, axs = plt.subplots(nrows, ncols, figsize=( 8 * ncols, 6 * nrows), facecolor='w', edgecolor='k')

    if nrows == 1:
        axs = axs.reshape(-1, ncols)  # Make sure it's a 2D array

    for i, prefix in enumerate(unique_prefixes):
        for j, label in enumerate(["UL", "DL", "EE"]):
            ax = axs[i, j]
            ax.set_title(f"{label} Latency", fontweight="bold", fontsize="14")
            ax.grid()

            legend_handles_colors = []

            for entry, meta in items:
                if str(meta['case'])[:3] == prefix:
                    df = DATA.get(entry, [])
                
                    latencies = {
                        "UL": latency_ul(df),
                        "DL": latency_dl(df),
                        "EE": latency_ee(df)
                    }
                    xs = latencies[label].sort_values()
                    cdf = xs.rank(pct=True) * 100
                    line, = ax.plot(cdf, xs, "-^", markersize=1, label=f"{return_label(str(meta['case']))}")
                    ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

                    # lines to x and y axis
                    p = xs.quantile(0.5)
                    line_50 = ax.axhline(p, color=line.get_color(), linestyle='--', linewidth=.8, label="50%")
                    p = xs.quantile(0.95)
                    line_95 = ax.axhline(p, color=line.get_color(), linestyle='-', linewidth=.8, label="95%")

                    legend_handles_colors.append(line)
            
            legend_handles_shapes = [
                Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
                Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
            ]

            ax.axvline(50, color="k", linestyle='--')
            ax.axvline(95, color="k")
            ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper left')

    fig.suptitle('CDFs of different message sizes and frequencies', fontsize=24, fontweight='bold')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_cdf_per_scenario_load" if PLOT_TAG else "latency_cdf"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### CDFs of all Test Cases combined

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: str(x[1]["case"]))
  
    nrows = 1
    ncols = 3  # UL, DL, and EE for each unique prefix

    fig, axs = plt.subplots(nrows, ncols, figsize=(8 * ncols, 8 * nrows), facecolor='w', edgecolor='k')

    if nrows == 1:
        axs = axs.reshape(-1, ncols)  # Make sure it's a 2D array

    i=0
    for j, label in enumerate(["UL", "DL", "EE"]):
        ax = axs[i, j]
        ax.set_title(f"{label} Latency", fontweight="bold", fontsize="14")
        ax.grid()

        for entry, meta in items:
            df = DATA.get(entry, [])
        
            latencies = {
                "UL": latency_ul(df),
                "DL": latency_dl(df),
                "EE": latency_ee(df)
            }
            xs = latencies[label].sort_values()
            cdf = xs.rank(pct=True) * 100

            ax.plot(cdf, xs, LINE_STYLES[int(str(meta['case'])[0])-1], markersize=0.5, label=f"{return_label(str(meta['case']))}", color=COLOR_PALETE[get_color_index(meta['case'])])
            ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

        ax.legend(loc='upper left')

    fig.suptitle("CDFs of all Test Cases combined", fontweight="bold", fontsize=24)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_cdf_all_cases" if PLOT_TAG else "latency_cdf_all_cases"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### CDFs (Baseline and Absolute Priority side by side)

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: (str(x[1]['case'])[1:], str(x[1]['case'])[0]))

    unique_prefixes = sorted(set(str(meta['case'])[1:] for _, meta in items))

    nrows = len(unique_prefixes)
    ncols = max([list([str(meta['case'])[1:] for _, meta in items]).count(prefix) for prefix in unique_prefixes])

    fw, fh = 6 * ncols, 6 * nrows
    fig, axs = plt.subplots(nrows, ncols, figsize=(fw, fh), facecolor='w', edgecolor='k')
    
    # Make sure axs is always a 2D array
    if not isinstance(axs, np.ndarray):
        axs = np.array([[axs]])

    if nrows == 1:
        axs = axs.reshape(nrows, -1)
    if ncols == 1:
        axs = axs.reshape(-1, ncols)

    current_row = 0
    current_col = 0
    last_prefix = None

    for entry, meta in items:
        legend_handles_colors = []
        current_prefix = str(meta['case'])[1:]
        
        if last_prefix is not None and current_prefix != last_prefix:
            current_row += 1
            current_col = 0
        
        ax = axs[current_row, current_col]

        df = DATA.get(entry, [])
        ul = latency_ul(df)
        dl = latency_dl(df)
        ee = latency_ee(df)

        ax.set_title(f"{info_label(meta)}", fontweight="bold", fontsize="14")
        ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

        if LATENCY_PLOT_MAX or LATENCY_PLOT_MIN:
            ax.set_ylim(LATENCY_PLOT_MIN, LATENCY_PLOT_MAX)

        ax.grid()

        for label, xs in zip(["UL", "DL", "EE"], [ul, dl, ee]):
            xs = xs.sort_values()
            cdf = xs.rank(pct=True) * 100
            line, = ax.plot(cdf, xs, "-^", label=label, markersize=4, color=COLORS[label])

            # lines to x and y axis
            p = xs.quantile(0.5)
            ax.axhline(p, color=COLORS[label], linestyle='--')
            # ax.annotate(f'{p:.02f}', xy=(2, p + 2), color='k')

            p = xs.quantile(0.95)
            ax.axhline(p, color=COLORS[label], linestyle='-')
            # ax.annotate(f'{p:.02f}', xy=(2, p + 2), color='k')
            
            legend_handles_colors.append(line)
        
        legend_handles_shapes = [
            Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
            Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
        ]
    
        ax.axvline(50, color="k", linestyle='--')
        ax.axvline(95, color="k")
        ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper left')

        current_col += 1
        last_prefix = current_prefix

    title = fig.suptitle("CDFs (Baseline and Absolute Priority side by side)", fontweight="bold", fontsize=24)
    title.set_y(0.95)  # Adjust this value to set the vertical position of the title
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_cdf_bl_ap_side_by_side" if PLOT_TAG else "latency_cdf_bl_ap_side_by_side"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### CDFs (Baseline and Absolute Priority under different loads)

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: str(x[1]["case"]))
    
    unique_prefixes = sorted(set(str(meta['case'])[3] for _, meta in items))
    nrows = len(unique_prefixes)
    ncols = 3  # UL, DL, and EE for each unique prefix

    fig, axs = plt.subplots(nrows, ncols, figsize=(8 * ncols, 8 * nrows), facecolor='w', edgecolor='k')

    if nrows == 1:
        axs = axs.reshape(-1, ncols)  # Make sure it's a 2D array

    for i, prefix in enumerate(unique_prefixes):
        for j, label in enumerate(["UL", "DL", "EE"]):
            legend_handles_colors = []
            ax = axs[i, j]
            ax.set_title(f"{label} Latency", fontweight="bold", fontsize="14")
            ax.grid()

            for entry, meta in items:
                if str(meta['case'])[3] == prefix and int(str(meta['case'])[1]) in LOADS_DISPLAYED[label]:
                    df = DATA.get(entry, [])
                
                    latencies = {
                        "UL": latency_ul(df),
                        "DL": latency_dl(df),
                        "EE": latency_ee(df)
                    }
                    xs = latencies[label].sort_values()
                    cdf = xs.rank(pct=True) * 100
                    line, = ax.plot(cdf, xs, LINE_STYLES[int(str(meta['case'])[0])-1], markersize=1, label=f"Test Case: {meta['case']}", color=COLOR_PALETE[get_color_index(meta['case'])])
                    legend_handles_colors.append(line)
                    ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

                    # lines to x and y axis
                    p = xs.quantile(0.5)
                    ax.axhline(p, color=line.get_color(), linestyle='-', linewidth=.8)
                    p = xs.quantile(0.95)
                    ax.axhline(p, color=line.get_color(), linestyle='--', linewidth=.8)

            legend_handles_shapes = [
                Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
                Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
            ]

            ax.axvline(50, color="k", linestyle='--')
            ax.axvline(95, color="k")
            ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper left')

    title = fig.suptitle("CDFs (Baseline and Absolute Priority under different loads)", fontweight="bold", fontsize=24)
    title.set_y(0.95)  # Adjust this value to set the vertical position of the title
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_cdf_bl_ap_different_loads" if PLOT_TAG else "latency_cdf_bl_ap_different_loads"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### Comparing CDFs of different loads

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: str(x[1]["case"]))
    
    unique_prefixes = sorted(set(str(meta['case'])[0:2] for _, meta in items))
    unique_last_prefixes = sorted(set(str(meta['case'])[3] for _, meta in items))
    nrows = len(unique_prefixes)*len(unique_last_prefixes)
    ncols = 3  # UL, DL, and EE for each unique prefix

    fig, axs = plt.subplots(nrows, ncols, figsize=(8 * ncols, 8 * nrows), facecolor='w', edgecolor='k')

    if nrows == 1:
        axs = axs.reshape(-1, ncols)  # Make sure it's a 2D array

    counter = 0
    for i, prefix in enumerate(unique_prefixes):
        for last_prefix in unique_last_prefixes:
            for j, label in enumerate(["UL", "DL", "EE"]):
                legend_handles_colors = []
                ax = axs[counter, j]
                ax.set_title(f"{label} Latency", fontweight="bold", fontsize="14")
                ax.grid()

                for entry, meta in items:
                    if str(meta['case'])[0:2] == prefix and str(meta['case'])[3] == last_prefix:
                        df = DATA.get(entry, [])
                    
                        latencies = {
                            "UL": latency_ul(df),
                            "DL": latency_dl(df),
                            "EE": latency_ee(df)
                        }
                        xs = latencies[label].sort_values()
                        cdf = xs.rank(pct=True) * 100
                        line, = ax.plot(cdf, xs, "-^", markersize=1, label=f"Test Case: {meta['case']}")
                        legend_handles_colors.append(line)

                    ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

                    # lines to x and y axis
                    p = xs.quantile(0.5)
                    ax.axhline(p, color=line.get_color(), linestyle='--', linewidth=.8)
                    p = xs.quantile(0.95)
                    ax.axhline(p, color=line.get_color(), linestyle='-', linewidth=.8)

                legend_handles_shapes = [
                    Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
                    Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
                ]

                ax.axvline(50, color="k", linestyle='--')
                ax.axvline(95, color="k")
                ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper left')
                 
            counter += 1                     

    title = fig.suptitle("Comparing CDFs of different loads", fontweight="bold", fontsize=24)
    title.set_y(0.95)  # Adjust this value to set the vertical position of the title
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_comparing_loads_and_features" if PLOT_TAG else "latency_comparing_loads_and_features"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### CDFs comparing features

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=lambda x: str(x[1]["case"])[-3:])  # Sorting by last three digits
    
    unique_prefixes = sorted(set(str(meta['case'])[-3:] for _, meta in items))

    nrows = len(unique_prefixes)
    ncols = 3  # UL, DL, EE

    fig, axs = plt.subplots(nrows, ncols, figsize=(8 * ncols, 8 * nrows), facecolor='w', edgecolor='k')

    if nrows == 1:
        axs = axs.reshape(-1, ncols)  # Make sure it's a 2D array

    for i, prefix in enumerate(unique_prefixes):
        for j, label in enumerate(["UL", "DL", "EE"]):
            legend_handles_colors = []
            ax = axs[i, j]
            ax.set_title(f"{label} Latency", fontweight="bold", fontsize="14")
            ax.grid()

            for entry, meta in items:
                if str(meta['case'])[-3:] == prefix:  # Filtering by last three digits
                    df = DATA.get(entry, [])
                
                    latencies = {
                        "UL": latency_ul(df),
                        "DL": latency_dl(df),
                        "EE": latency_ee(df)
                    }
                    xs = latencies[label].sort_values()
                    cdf = xs.rank(pct=True) * 100
                    line, = ax.plot(cdf, xs, "-^", markersize=1, label=f"Test Case: {meta['case']}", color=COLOR_PALETE[int(str(meta['case'])[0])-1])
                    legend_handles_colors.append(line)
                    ax.set_ylabel(f"Latency [ms]", fontweight="bold", fontsize="14")

                    # lines to x and y axis
                    p = xs.quantile(0.5)
                    ax.axhline(p, color=line.get_color(), linestyle='-', linewidth=.8)
                    p = xs.quantile(0.95)
                    ax.axhline(p, color=line.get_color(), linestyle='--', linewidth=.8)

            legend_handles_shapes = [
                Line2D([0], [0], color='k', lw=1, linestyle='--', label="50%"), 
                Line2D([0], [0], color='k', lw=1, linestyle='-', label='95%')
            ]

            ax.axvline(50, color="k", linestyle='--')
            ax.axvline(95, color="k")
            ax.legend(handles=legend_handles_colors + legend_handles_shapes, loc='upper left')

    title = fig.suptitle("CDFs comparing features", fontweight="bold", fontsize=24)
    title.set_y(0.95)  # Adjust this value to set the vertical position of the title
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_comparing_features" if PLOT_TAG else "latency_comparing_features"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

### Handover Events Plot

In [ ]:
CURRENT_ANTENNA = 0  # Start with the first antenna

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=sort_key)
    
    unique_prefixes = sorted(set(get_row_identifier(meta['case']) for _, meta in items))

    nrows = len(unique_prefixes)
    ncols = max([list(map(int, [get_row_identifier(meta['case']) for _, meta in items])).count(prefix) for prefix in unique_prefixes])

    fw, fh = 8 * ncols, 6 * nrows
    fig, axs = plt.subplots(nrows, ncols, figsize=(fw, fh), facecolor='w', edgecolor='k')

    # Make sure axs is always a 2D array
    if not isinstance(axs, np.ndarray):
        axs = np.array([[axs]])

    if nrows == 1:
        axs = axs.reshape(nrows, -1)
    if ncols == 1:
        axs = axs.reshape(-1, ncols)

    current_row = 0
    current_col = 0
    last_prefix = None
    
    for entry, meta in items:
        current_prefix = get_row_identifier(meta['case'])
        current_antenna = CURRENT_ANTENNA  # reset for next plot
        HO_FLAG = False

        if last_prefix is not None and current_prefix != last_prefix:
            current_row += 1
            current_col = 0
        
        ax = axs[current_row, current_col]
        df = DATA.get(entry, [])
        idxs = []  

        if 'lon' not in df.keys():  break    
        
        if entry in HANDOVER_TIMES:
            # New code to mark handover positions
            for handover_times in HANDOVER_TIMES[entry]:
                
                start_ho, end_ho = find_handover_coordinates(df, handover_times, CORRECT_CLOCK_OFFSET)
                idxs.append([start_ho[2], end_ho[2]])
                
                ax.plot([start_ho[0], end_ho[0]], [start_ho[1], end_ho[1]], color=COLORS['HOs'], zorder=2)
                ax.axvline(start_ho[0], color="k", linestyle='--', linewidth=.2)
                ax.axhline(start_ho[1], color="k", linestyle='--', linewidth=.2)
                ax.axvline(end_ho[0], color="k", linestyle='-', linewidth=.2)
                ax.axhline(end_ho[1], color="k", linestyle='-', linewidth=.2)

            for idx in range(len(df) - 1):
                start = (df['lon'].iloc[idx], df['lat'].iloc[idx])
                end = (df['lon'].iloc[idx + 1], df['lat'].iloc[idx + 1])
                
                # Determine color based on whether the time is within handover
                if is_within_handover(df, idx, HANDOVER_TIMES[entry], idxs, CORRECT_CLOCK_OFFSET):
                    color = COLORS['HOs']
                    HO_FLAG = True
                else:
                    if HO_FLAG: current_antenna = 1 - current_antenna  # Switch antennas at handover
                    color = COLORS[f'Antenna{current_antenna+1}']
                    HO_FLAG = False

                ax.plot([start[0], end[0]], [start[1], end[1]], color=color, zorder=2)
        else:
            ax.plot(df['lon'], df['lat'], color=COLORS['Trajectory'], zorder=2)

        # Add red dots to indicate where antennas are
        for i, antenna in enumerate(ANTENNAS_COORDINATES):
            ax.scatter(antenna[0], antenna[1], marker='s', color=COLORS[f'Antenna{i+1}'], zorder=2, s=100)

        max_x, min_x = df['lon'].max(), df['lon'].min()
        max_y, min_y = df['lat'].max(), df['lat'].min()
        ax.set_ylim([min_y-0.0005, max_y+0.0005])
        ax.set_xlim([min_x-0.0008, max_x+0.0005])
        
        # ax.set_ylim([59.403323666700246, 59.40501911555832])
        # ax.set_xlim([17.95147451245423, 17.95552101792219])

        ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True, useOffset=False))
        ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True, useOffset=False))
        ax.ticklabel_format(style='plain')  # Turns off scientific notation


        ax.set_title(f" Trajectory of test case: \n {info_label(meta)}", fontweight="bold", fontsize="14")
        ax.set_ylabel(f"Latitude", fontweight="bold", fontsize="14")
        ax.set_xlabel(f"Longitude", fontweight="bold", fontsize="14")
        ax.grid()

        legend_elements = [
            Patch(facecolor=COLORS['Antenna1'], edgecolor='black', label='Antenna 1'),
            Patch(facecolor=COLORS['Antenna2'], edgecolor='black', label='Antenna 2'),
            Patch(facecolor=COLORS['HOs'], edgecolor='black', label='Handover')
        ]

        ax.legend(handles=legend_elements, loc='upper right') 

        current_col += 1
        last_prefix = current_prefix

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__handover_plot" if PLOT_TAG else "handover_plot"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()

In [ ]:
def main():
    
    fw, ncols = 6, 2
    fh, nrows = 4, len(META)
    fig = plt.figure(figsize=(fw*ncols, fh*nrows), facecolor="w", edgecolor="k")
    axs = fig.subplots(nrows, ncols, sharey='row').reshape(nrows, ncols)
    
    items = list(META.items())
    items.sort(key=lambda x: x[1]["case"])
    for i, (entry, meta) in enumerate(items):
        df = DATA[entry]
        idxs = []

        if entry in HANDOVER_TIMES:
            for handover_times in HANDOVER_TIMES[entry]:
                start_ho, end_ho = find_handover_coordinates(df, handover_times, CORRECT_CLOCK_OFFSET)
                idxs.append([start_ho[2], end_ho[2]])
        
            for j, title in enumerate(["UL packets", "DL packets"]):
                ax = axs[i, j]
                current_antenna = CURRENT_ANTENNA  # reset for next plot
                HO_FLAG = False

                ax.set_title(title, fontweight="bold")
                axs[i, 0].set_ylabel(f"{info_label(meta)}\n{return_label(str(meta['case']))}\n\nLatency [ms]")

                if LATENCY_PLOT_MAX or LATENCY_PLOT_MIN:
                    ax.set_ylim(LATENCY_PLOT_MIN, LATENCY_PLOT_MAX)
                ax.grid()

                if j == 0:
                    ul = latency_ul(df)
                    for idx in range(len(ul) - 1):
                        if is_within_handover(df, idx, HANDOVER_TIMES[entry], idxs, CORRECT_CLOCK_OFFSET):
                            color = COLORS['HOs']
                            HO_FLAG = True
                        else:
                            if HO_FLAG: current_antenna = 1 - current_antenna  # Switch antennas at handover
                            HO_FLAG = False
                            color = COLORS[f'Antenna1{"_Light" if current_antenna else ""}']
                            
                        ax.plot([idx, idx+1], [ul.iloc[idx], ul.iloc[idx+1]], color=color, zorder=2, linewidth=0.7)
                    # ax.plot(ul)
                    ax.set_xlabel("Packet number")
                if j == 1:
                    dl = latency_dl(df)
                    for idx in range(len(dl) - 1):
                        if is_within_handover(df, idx, HANDOVER_TIMES[entry], idxs, CORRECT_CLOCK_OFFSET):
                            color = COLORS['HOs']
                            HO_FLAG = True
                        else:
                            if HO_FLAG: current_antenna = 1 - current_antenna  # Switch antennas at handover
                            HO_FLAG = False
                            color = COLORS[f'Antenna{current_antenna+1}']
                            
                        ax.plot([idx, idx+1], [dl.iloc[idx], dl.iloc[idx+1]], color=color, zorder=2, linewidth=0.8)
                    # ax.plot(dl)
                    ax.set_xlabel("Packet number")

            legend_elements = [
                Patch(facecolor=COLORS['Antenna1'], edgecolor='black', label='Antenna 1'),
                Patch(facecolor=COLORS['Antenna2'], edgecolor='black', label='Antenna 2'),
                Patch(facecolor=COLORS['HOs'], edgecolor='black', label='Handover')
            ]

            ax.legend(handles=legend_elements, loc='upper right') 

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    filename = f"{PLOT_TAG}__handover_event_packet_numbers" if PLOT_TAG else "handover_event_packet_numbers"
    fig.savefig(PLOTS_DIR / f"{filename}.png")

main()

### Latency vs Velocity

In [ ]:
def main():
    items = list(META.items())
    items.sort(key=sort_key)
    
    unique_prefixes = sorted(set(get_row_identifier(meta['case']) for _, meta in items))

    nrows = len(unique_prefixes)
    ncols = max([list(map(int, [get_row_identifier(meta['case']) for _, meta in items])).count(prefix) for prefix in unique_prefixes])

    fw, fh = 8 * ncols, 6 * nrows
    fig, axs = plt.subplots(nrows, ncols, figsize=(fw, fh), facecolor='w', edgecolor='k')

    # Make sure axs is always a 2D array
    if not isinstance(axs, np.ndarray):
        axs = np.array([[axs]])

    if nrows == 1:
        axs = axs.reshape(nrows, -1)
    if ncols == 1:
        axs = axs.reshape(-1, ncols)

    current_row = 0
    current_col = 0
    last_prefix = None

    for entry, meta in items:
        current_prefix = get_row_identifier(meta['case'])
        if last_prefix is not None and current_prefix != last_prefix:
            current_row += 1
            current_col = 0
        
        ax = axs[current_row, current_col]
        
        df = DATA.get(entry, [])
        ul = latency_ul(df)
        dl = latency_dl(df)
        # ee = latency_ee(df)

        if 'vel' in df:
            # Create plot - if you want to check how barplot look change scatter to bar
            for label, latency in ({'UL': ul, 'DL': dl}).items():
                ax.scatter(df['vel'], latency, alpha=0.3, color=COLORS[label])
          
        ax.set_title(f"Test case: \n {info_label(meta)}", fontweight="bold", fontsize="14")
        ax.set_ylabel(f"Latency(ms)", fontweight="bold", fontsize="14")
        ax.set_xlabel(f"Velocity", fontweight="bold", fontsize="14")
        ax.grid()
        ax.legend(['Uplink', 'Downlink', 'End2end'], loc='upper right', fontsize="14") 

        current_col += 1
        last_prefix = current_prefix

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    filename = f"{PLOT_TAG}__latency_velocity" if PLOT_TAG else "latency_velocity"
    fig.savefig(f"{PLOTS_DIR}/{filename}.png")

if __name__ == "__main__":
    main()